In [3]:
import sys
sys.path.append('../ml-fastvit')

In [4]:
import timm
import torch
import torch.optim as optim
import torchvision.utils as vutils
import torchvision.transforms as T
import torch.nn as nn
import models
import math
import matplotlib.pyplot as plt
import numpy as np
import gc

from torch import Tensor
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from timm.models import create_model
from models.modules.mobileone import reparameterize_model
from PIL import Image
from urllib.request import urlopen

# from UpsampleTransformerDecoder import UpsampleTransformerDecoder
# from UpsampleTransformerDecoderV2 import UpsampleTransformerDecoderV2


# Before starting the training, make sure to clear any residual memory
gc.collect()
torch.cuda.empty_cache()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [5]:
def imshow(img, mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)):
    """Imshow for Tensor."""
    img = img.numpy().transpose((1, 2, 0))
    # img = std * img + mean  # unnormalize
    # img = np.clip(img, 0, 1)  # clip any values outside the range [0, 1]
    plt.imshow(img)
    plt.show()

In [6]:
transform = T.Compose([
        T.Resize((64, 64)),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [7]:
# To Train from scratch/fine-tuning
encoder = create_model("fastvit_t8", fork_feat=True) # can turn fork_feat to False

# checkpoint = torch.load('./pretrained/fastvit_t8.pth.tar')
# encoder.load_state_dict(checkpoint['state_dict'], strict=False)

# For inference
# model.eval()      
encoder = reparameterize_model(encoder)
encoder = encoder.to(device)
encoder.eval()

FastViT(
  (patch_embed): Sequential(
    (0): MobileOneBlock(
      (se): Identity()
      (activation): GELU(approximate='none')
      (reparam_conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): MobileOneBlock(
      (se): Identity()
      (activation): GELU(approximate='none')
      (reparam_conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=48)
    )
    (2): MobileOneBlock(
      (se): Identity()
      (activation): GELU(approximate='none')
      (reparam_conv): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (network): ModuleList(
    (0): Sequential(
      (0): RepMixerBlock(
        (token_mixer): RepMixer(
          (reparam_conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48)
        )
        (convffn): ConvFFN(
          (conv): Sequential(
            (conv): Conv2d(48, 48, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=48, bias=False)
            (bn): 